In [1]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import pandas as pd
import warnings
import time

# Uyarıları kapat
warnings.filterwarnings("ignore")

# Zaman damgası başlat
seconds = time.time()

# Özellik listesi ve etiket sütunu
features = [
    "Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
    "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total", "Flow Duration", "Bwd Packet Length Max", "Flow IAT Max",
    "Flow IAT Mean", "Total Length of Bwd Packets", "Fwd Packet Length Min", "Bwd Packet Length Mean",
    "Flow Packets/s", "Fwd Packet Length Mean", "Total Backward Packets", "Total Fwd Packets",
    "Fwd Packet Length Max", "Bwd Packet Length Min", "Label"
]

# Veri seti yükleme
df = pd.read_csv('all_data.csv', usecols=features).fillna(0)

# Etiket sütununu 0 (Attack) ve 1 (BENIGN) olarak dönüştür
df['Label'] = df['Label'].apply(lambda x: 1 if x == "BENIGN" else 0)
y = df['Label']

# Sonuç çıktısı başlık
print('{:<20} {:<30} {:<10} {:<10} {:<15}'.format("ML Algorithm", "Feature Name", "F1-score", "Accuracy", "Feature List"))

# Makine öğrenimi algoritmaları
ml_list = {
    "Naive Bayes": GaussianNB(),
    "QDA": QDA(),
    "MLP": MLPClassifier(hidden_layer_sizes=(13, 13, 13), max_iter=500)
}

# F-measure threshold
least = 0

# Algoritma üzerinde döngü
for ml_name, clf in ml_list.items():
    feature_list = []  # Seçilen en iyi özellikleri saklar
    for feature in features[:-1]:  # "Label" hariç tüm özellikleri gez
        feature_list.append(feature)
        X = df[feature_list]  # Seçilen özelliklere göre veri setini hazırla

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

        # Modeli eğit ve tahmin yap
        clf.fit(X_train, y_train)
        predict = clf.predict(X_test)

        # Performans metrikleri
        f1 = f1_score(y_test, predict, average='macro')
        accuracy = round(clf.score(X_test, y_test), 2)

        # Özellik listesini indeks olarak dönüştür
        temp_features = "[" + ", ".join([str(features.index(f) + 1) for f in feature_list]) + "]"

        # F1 skorunu kontrol et
        if f1 >= least:
            least = f1
            print('{:<20} {:<30} {:<10.2f} {:<10.2f} {:<15} {}'.format(
                ml_name, feature, f1, accuracy, temp_features, "------> New feature found!!!"
            ))
        else:
            feature_list.pop()  # F1 skoru artmazsa özelliği kaldır
            print('{:<20} {:<30} {:<10.2f} {:<10.2f} {:<15}'.format(
                ml_name, feature, f1, accuracy, temp_features
            ))

    # Algoritma için en iyi özellik listesi ve en yüksek F1 skoru
    print(f"\nF1={least:.2f} {ml_name} The most efficient feature list = {feature_list}\n\n")

# Program sonu mesajı
print("Mission accomplished!")
print(f"Operation time: {time.time() - seconds:.2f} seconds")


ML Algorithm         Feature Name                   F1-score   Accuracy   Feature List   
Naive Bayes          Bwd Packet Length Std          0.73       0.88       [1]             ------> New feature found!!!
Naive Bayes          Flow Bytes/s                   0.67       0.86       [1, 2]         
Naive Bayes          Total Length of Fwd Packets    0.73       0.88       [1, 3]         
Naive Bayes          Fwd Packet Length Std          0.72       0.88       [1, 4]         
Naive Bayes          Flow IAT Std                   0.70       0.86       [1, 5]         
Naive Bayes          Flow IAT Min                   0.72       0.88       [1, 6]         
Naive Bayes          Fwd IAT Total                  0.65       0.83       [1, 7]         
Naive Bayes          Flow Duration                  0.65       0.83       [1, 8]         
Naive Bayes          Bwd Packet Length Max          0.71       0.87       [1, 9]         
Naive Bayes          Flow IAT Max                   0.68       0.85    